In [168]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [169]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [170]:
#read all the data:
items = pd.read_csv("dressipi_recsys2022/item_features.csv")
candidate = pd.read_csv("dressipi_recsys2022/candidate_items.csv")
purchase =  pd.read_csv("dressipi_recsys2022/train_purchases.csv")
sessions =  pd.read_csv("dressipi_recsys2022/train_sessions.csv")
sessions_test = pd.read_csv("dressipi_recsys2022/test_final_sessions.csv")
leaderboard = pd.read_csv("dressipi_recsys2022/test_leaderboard_sessions.csv")
#set the date column to a date format
purchase['date'] = pd.to_datetime(purchase['date'])

In [171]:
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]
#top 99 most sold items in the last month of the training data
top_seller = new_purchases['item_id'].value_counts()[0:100].index.to_list()


In [190]:
sessions = sessions.sample(frac = 0.2)


In [191]:
#uinique session numbers and items
sessions_unique = sessions.session_id.unique()
items_unique = items.item_id.unique()
len(items_unique)

23691

**Vectorized dictionary of items:**


In [192]:
item_charac = {}
for i in tqdm(items_unique):
    selected_items = items[items.item_id == i]
    llista = []
    for j in range(len(selected_items)):
        llista.append(selected_items.iloc[j,1])
        llista.append(selected_items.iloc[j,2])
    item_charac[i] = llista

100%|██████████| 23691/23691 [00:49<00:00, 480.89it/s]


In [193]:
len(sessions)

948764

**Items seen in each session**:

In [194]:
session_dict = {}
past_session = 3
llista =[]
for i in tqdm(range(len(sessions))):
    actual_session = sessions.iloc[i].session_id
    if actual_session != past_session:
        session_dict[past_session] = llista
        past_session = actual_session
        llista = [sessions.iloc[i].item_id]
    else: 
        llista.append(sessions.iloc[i].item_id)
    

session_dict
    

100%|██████████| 948764/948764 [03:20<00:00, 4721.11it/s]


{3: [9655],
 895571: [3325],
 1147421: [21993],
 2489721: [12804],
 631831: [5991],
 3056680: [15923],
 4314766: [17479],
 394951: [18156],
 2745743: [11923],
 3066749: [6454],
 689186: [15041],
 3816121: [12665],
 876334: [16774],
 749448: [11271],
 3741950: [26419],
 1769643: [27395],
 1232976: [9594],
 4366067: [11186],
 809929: [893],
 1062371: [24222],
 2572541: [27988],
 3044972: [2410],
 4184915: [333],
 661829: [3392],
 1773473: [19788],
 2045340: [23764],
 1773718: [17774],
 643952: [4760],
 4063183: [1474],
 1416223: [15501],
 3672106: [222],
 2150569: [12401],
 4425373: [5005],
 2149664: [13709],
 1336204: [4768],
 2530372: [22719],
 1395901: [158],
 2018042: [14379],
 685980: [203],
 3712590: [12842],
 2444648: [26379],
 4392602: [5691],
 2405513: [11737],
 1900497: [26199],
 3316106: [10716],
 892845: [9608],
 1161325: [14541],
 339168: [26198],
 145591: [18007],
 1173455: [11824],
 2116432: [4871],
 2273041: [22093],
 3754953: [2021],
 1719496: [1065],
 663322: [16963],
 

In [175]:
'''
session_dict = {}
for i in tqdm(sessions_unique):
    llista = []
    actual_session = sessions[sessions.session_id == int(i)]
    for j in range(len(actual_session)):
        llista.append(actual_session.iloc[j,1])
    session_dict[i] = llista
'''
    

  0%|          | 354/1000000 [00:03<2:35:16, 107.29it/s]


KeyboardInterrupt: 

**List of similar items:**

In [ ]:
def make_pairs(x, y):
    """Creates a tuple containing image pairs with corresponding label.

    Arguments:
        x: List containing images, each index in this list corresponds to one image.
        y: List containing labels, each label with datatype of `int`.

    Returns:
        Tuple containing two numpy arrays as (pairs_of_samples, labels),
        where pairs_of_samples' shape is (2len(x), 2,n_features_dims) and
        labels are a binary array of shape (2len(x)).
    """
 
    num_classes = 23691
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

    pairs = []
    labels = []

    for idx1 in range(len(x)):
        # add a matching example
        x1 = x[idx1]
        label1 = y[idx1]
        idx2 = random.choice(digit_indices[label1])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [1]

        # add a non-matching example
        label2 = random.randint(0, num_classes - 1)
        while label2 == label1:
            label2 = random.randint(0, num_classes - 1)

        idx2 = random.choice(digit_indices[label2])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [0]

    return np.array(pairs), np.array(labels).astype("float32")


# make train pairs
pairs_train, labels_train = make_pairs(x_train, y_train)

# make validation pairs
pairs_val, labels_val = make_pairs(x_val, y_val)

# make test pairs
pairs_test, labels_test = make_pairs(x_test, y_test)

In [142]:
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))


input = layers.Input((28, 28, 1))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(4, (5, 5), activation="tanh")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(16, (5, 5), activation="tanh")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(10, activation="tanh")(x)
embedding_network = keras.Model(input, x)


input_1 = layers.Input((28, 28, 1))
input_2 = layers.Input((28, 28, 1))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)

NameError: name 'y' is not defined